SCRAPING - ss 21 
--------------- 
> Chiara Piccolroaz

> University of Konstanz

> Department of Computer and Information Science; Department of political and administration science


IMPORTING PACKAGES
--------------------------

WEbseiten for Scraping Juris : https://www.juris.de/jportal/portal/ and https://rzblx10.uni-regensburg.de/dbinfo/detail.php?bib_id=ubko&colors=&ocolors=&lett=f&tid=0&titel_id=865

WebSeite for Installing Selenium: : https://www.jcchouinard.com/learn-selenium-python-seo-automation/

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
import requests, PyPDF2
from io import BytesIO
import pickle 

BEFOR STARTING:

 Make sure to: (1) be locked in the Uni-WLAN or connected with the Uni-VPN. (2) the packages have been download

IDEE OF THE PROJEKT: 

It is for us impossible to download the PDF, which we need (is there maybe another way?). For this reason, we first extract the PDFs-links (Part1), so that we can use another function (Part2), to reopen the PDFs, extract the text and save it in a text-file. 

PROBLEME: 

(1) Ich werde nach ein paar Stunden Scraping automatisch von der Website abgemeldet, was dazu führt, dass der ganze Prozess aufhört und die Webseite nich nicht merken kann, welche die letzte Operation war. 
(2) Part 2 works only on not-Juris PDFs

PART 1. Get Links
-------------------------

Open the Web

In [3]:
driverC=webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe") 
driverC.get('https://www.juris.de/jportal/?action=JLoginUser&username=HEBISUKN.autologin')
driverC.get('https://www.juris.de/r3/search')
driverC.maximize_window()
pickle.dump(driverC.get_cookies() , open("QuoraCookies.pkl","wb"))

for cookie in pickle.load(open("QuoraCookies.pkl", "rb")): 
    driverC.add_cookie(cookie)

Select the relevant filers and buttons. Make sure to: run the sleep functions, otherwise the other functions will override. 


In [4]:
button = driverC.find_element_by_id("categories_hits_Rechtsprechung")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Sachgebiete")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Asylis___MILo")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_apply")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("linkWorkId80896")
button.click()
time.sleep(2) 

Collect the links: 
Collect the link of the pdfs for each of the 19151 elements. In order to do that use the following function, which enters each of the 19151 element files, get all PDFs-links and save them in a list. 

We now try for 10 PDFs. The number 10 must be replaced by 19151 when running the projekt for real

In [5]:
# 0. Initialisation

# the list the PDF-files are gonna be saved in
listPDFs = [] 
# i refers to the number of ducuments pro page and gives the function the id of the document we are looking for. Put i = j = 0
i = 0 
# j refers to the number of documents in total. Put i = j = 0
j = 0
# p refers to the number of the page 
p = 1

# 0. repeat the function for the number of the documents: 
while j < 10:
    # get the name
    # create the string, which indicates the element_id on the website, which we want to enter the element file with 
    str1 ='resultlistentry_LI' 
    str2 = "{}".format(i)
    str = str1 + str2
    print(str) 
    
# 1. enter the element´s file
    Urteil = driverC.find_element_by_id(str) 
    Urteil.click()
    time.sleep(1) 
    
# 2. get the PDF-link(s) from the respective element´s file
    # get the links and put in the list linkList
    links=driverC.find_elements_by_tag_name("a")
    linkList = [link.get_attribute('href') for link in links]
    time.sleep(1)
    # clean the list linkList, in order to have only strings
    clean_linkList = [i for i in linkList if i is not None]
    # count how many PDFs are, which we are interested in. 
    # According to that we have 3 different functions to save the links and the texts of the PDF 
    count = 0
    sub = '/jportal/docs/anlage/as/bilder/'
    for link in clean_linkList:
        if sub in link:
            count = count +1
    # if these are more than just one PDF: 
    ### select the links, save them as list linksPDF and add it to the list linkPDF
    linkPDF = []
    TextList = []
    if count > 1:
        print('multiple PDFs')
        for link in clean_linkList:
            if sub in link:
                linkPDF.append(link)      
    
    #if there is only one PDF
    ### select the link and add to the list linkPDF;       
    if count == 1:
        print("got the PDFs, go one back")
        for link in clean_linkList:
            if sub in link:
                linkPDF = link
    
    #if there are no PDFs: do nothing  
    if count < 1:
        print('no PDF') 
        
# 4. add the PDF(s) in the ListPDFs
    listPDFs.append(linkPDF)
    #print(listPDFs)
    time.sleep(1)
    
# 5. go back to the main page and go on with the other elements
    backtoList = driverC.find_element_by_class_name("tlrahmen")
    backtoList.click()
    time.sleep(3) 
    
# 6. increase i to check for the elements pro page; increase j to check for the 19151 elements
    i = i+1
    j = j+1 
    
# 6. if te last element change the page 
    if i == 25: 
        print("i is equal 25 - we got the 25th element of the page -, so change page")  
        p = p+1
        print("NEXT page" , p)
        driverC.execute_script("window.scrollTo(0, 0);")
        button = driverC.find_element_by_class_name("svg-icon-chevron_right")
        button.click()
        time.sleep(3)  
        i= 0
        

resultlistentry_LI0
no PDF
resultlistentry_LI1
no PDF
resultlistentry_LI2
no PDF
resultlistentry_LI3
got the PDFs, go one back
resultlistentry_LI4
got the PDFs, go one back
resultlistentry_LI5
got the PDFs, go one back
resultlistentry_LI6
got the PDFs, go one back
resultlistentry_LI7
no PDF
resultlistentry_LI8
got the PDFs, go one back
resultlistentry_LI9
got the PDFs, go one back


With this function we want to restructure the list of links, by modifing the sublists: that means eleminating the empty  sublistst and ectracting the links from the sublists which containingmultiple informations. 

In [6]:
# eleminating the empty  sublistst
listPDFs_cleaned1 = [x for x in listPDFs if x]

# exctracting the links from the sublists which connatins multiple informations
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

listPDFs_cleaned2 = flatten(listPDFs_cleaned1)
listPDFs_cleaned = listPDFs_cleaned2

In [7]:
listPDFs_cleaned

['https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18455390.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17130144.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16634543.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16387029.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/15207340.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/19439387.pdf']

We can save the links into a text file

In [8]:
with open('Link.txt', 'w') as f:
    for item in listPDFs_cleaned:
        f.write("%s\r\n" % item)

PART 2: Get PDFs
--------------------------
This function can extract the text of PDFs which have been saved in a list, in our case in the list listPDFs_cleaned. Sadly in does not work for Juris-PDFs, but only for normal PDFs (You can run an example, with another link to a pdf)

In [9]:
# Diese Funktion fuktionier nur mit nicht-Juris-PDFs
TextList = []

for link in listPDFs_cleaned:
    Pages = []
    response = requests.get(link)
    my_raw_data = response.content
    
    with BytesIO(my_raw_data) as data:
        read_pdf = PyPDF2.PdfFileReader(data)
        
        for page in range(read_pdf.getNumPages()):
            p =read_pdf.getPage(page).extractText()
            p = p.replace("\n", "")
            Pages.append(p)
        JointPages = "".join(Pages)
        TextList.append(JointPages)

PdfReadError: Cannot read an empty file

We can save the texts of the PSDFs into a text file

In [ ]:
with open('test.txt', 'w') as f:
    for item in TextList:
        f.write("%s\r\n" % item)